item-based Collaborative Filtering for e-commerce
===

The goal is to find a model that gives the set of sku recommendations that a customer may buy

To test, given the test SKUs that have been bought, how many were in the prediction?

First thing to do is to connect to the dmbrandcommerce data base

In [2]:
import psycopg2
import pandas as pd
import statsmodels.api as sm
conn=psycopg2.connect(host="dmbrandcommerce.acommerce.platform"
                      ,database="dmbrandcommerce",
                      user="allen", password="8b5uAJjM")

Next is to get raw data, showing all customers and their purchases. We only need the customer ID and SKUs bought. In this case, we test with L'Oreal Thailand with 300k customers and 1300 SKUs. We get the customers with more than 4 unique purchases

In [27]:
from scipy import stats
import numpy as np
query = "WITH list AS (SELECT DISTINCT     COALESCE(shipping_phone,shipping_name) customer,     LOWER(COALESCE(acomm_category_2,acomm_category_1)) AS category  FROM datawarehouse.sat_sales_trx  JOIN datawarehouse.dm_product_attr1 USING(sku,partner_id)  WHERE partner_id=489 AND product_type IN ('ItemMaster','Set')    AND COALESCE(acomm_category_2,acomm_category_1) IS NOT NULL  ), main AS   (  SELECT     customer,     category,     COUNT(category) OVER(PARTITION BY customer) AS row1   FROM list  ) SELECT  customer,   category,   1 AS one FROM main WHERE row1>=5"
df=pd.read_sql_query(query,con=conn)
df

,customer,category,one
0,0617240347,body skincare,1
1,0617240347,body wash,1
2,0617240347,bundle,1
3,0617240347,conditioner,1
4,0617240347,shampoo,1
...,...,...,...
52137,989656155,bundle,1
52138,989656155,hand sanitizer,1
52139,989656155,undefined,1
52140,989656155,shampoo,1


Next is to pivot the table into customers and skus

In [28]:
category_bought = df.pivot_table(index = ['customer'],columns = ['category'], values = ['one'])['one']
category_bought

category,body skincare,body wash,bundle,conditioner,deodorant,detergent,dishwash,fabric softener,facial cleanser,facial skincare,hair treatment,hand sanitizer,remover,shampoo,undefined
customer,,,,,,,,,,,,,,,
+66635935898,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
0617240347,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
0628460635,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
0629964953,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
0634461588,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66999502618,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
66999759431,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0,NaN
864765721,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0


We will use the correlation between the Body Skincare

In [30]:
category_bought = category_bought.fillna(0)

In [31]:
category_bought['body skincare']

customer
+66635935898    1.0
0617240347      1.0
0628460635      1.0
0629964953      0.0
0634461588      1.0
               ... 
66999502618     0.0
66999759431     0.0
864765721       1.0
953651779       0.0
989656155       1.0
Name: body skincare, Length: 9403, dtype: float64

In [33]:
category_bought.corrwith(category_bought['body skincare'])

category
body skincare      1.000000
body wash         -0.041539
bundle            -0.051050
conditioner       -0.135706
deodorant         -0.032874
detergent         -0.020525
dishwash          -0.010436
fabric softener   -0.008457
facial cleanser   -0.068668
facial skincare   -0.052016
hair treatment    -0.106311
hand sanitizer    -0.033574
remover           -0.010127
shampoo           -0.037952
undefined         -0.042772
dtype: float64

In [24]:
similarCategories = category_bought.corrwith(category_bought['Body skincare'])
df2 = pd.DataFrame(similarCategories)
df2 = df2.dropna()
#df.head(10)
df2.head(10)

,0
category,
